# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902001


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:05<01:21,  2.73s/it]

Rendering models:  12%|█▎        | 4/32 [00:07<01:00,  2.17s/it]

Rendering models:  19%|█▉        | 6/32 [00:07<00:40,  1.54s/it]

Rendering models:  22%|██▏       | 7/32 [00:07<00:28,  1.13s/it]

...

Rendering models:  88%|████████▊ | 28/32 [00:10<00:00,  6.44it/s]

Rendering models:  91%|█████████ | 29/32 [00:10<00:00,  6.63it/s]

Rendering models:  94%|█████████▍| 30/32 [00:10<00:00,  6.79it/s]

Rendering models:  97%|█████████▋| 31/32 [00:11<00:00,  4.80it/s]

equidad1                              0.029100
not-logged-in-e0d22fca112ab22e0f7f    0.002018
Melina_t                              0.000508
elizabeth_s                           0.000673
kenyandall                            0.003874
not-logged-in-60953d21f1d9f596b2fe    0.000366
amerkg11                              0.000851
not-logged-in-0965bc82e356a9e112d3    0.018739
not-logged-in-3c24689ca793562b6e1b    0.001070
werjulia                              0.050857
Tommy80452                            0.001115
not-logged-in-f2eb212bec3c0ec6797b    0.029500
lsautter                              0.000611
not-logged-in-22877e3a7af4aafb750a    0.001819
not-logged-in-9a45911eb71184fe7c7f    0.001686
Phoebe14                              0.000417
egfresearch                           0.170056
jordanrushworth                       0.001090
not-logged-in-a4bed12d9111c0c34e9e    0.001093
Bambura                               0.000913
Fla                                   0.001325
beloaar      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())